In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook"
%matplotlib inline
plt.style.use('ggplot')

In [ ]:
from IPython.display import HTML
from ipywidgets.widgets import interact, IntSlider, FloatSlider, Layout
style = {'description_width': '100px'}
slider_layout = Layout(width='99%')

In [ ]:
#NOTE: those initial conditions do not represent any physical meaning
initN = 2930000
initS = 2920097
initE = 1
initIc = 1
initIsc = 1
initR = 0
beta = 1.52
delta = 0.04
kappa = 0.7
gamma = 0.62
roe = 0.7
alpha = 0.4
days = 200
initial_conditions = (initE, initIc, initIsc, initR, initN)
params = beta, alpha, roe, kappa, gamma, delta  

In [ ]:
def ode_model(z, t, beta, alpha, roe, kappa, gamma, delta): 
    S, E, Ic, Isc, R= z
    N = S + E + Ic + Isc + R 
    
    dSdt = -beta*S*(Ic+alpha*Isc)/N  + delta*R
    dEdt = beta*S*(Ic+alpha*Isc)/N - kappa*E
    dIcdt = roe*kappa*E - gamma*Ic
    dIscdt = (1-roe)*kappa*E - gamma*Isc
    dRdt = gamma*(Ic+Isc) - delta*R
    
    return [dSdt, dEdt, dIcdt, dIscdt, dRdt] 

In [ ]:
def ode_solver(t, initial_conditions, params):
    
    res = odeint(ode_model, [initS, initE, initIc, initIsc, initR], t, args=(beta, alpha, roe, kappa, gamma, delta))
    return res

In [ ]:
def main(initE, initIc, initIsc, initR, initN, beta, alpha, roe, kappa, gamma, delta, days):
    
    tspan = np.arange(0, days, 1)
    sol = ode_solver(tspan, initial_conditions, params)
    S, E, Ic, Isc, R = sol[:, 0], sol[:, 1], sol[:, 2], sol[:, 3], sol[:, 4]
    
    # Create traces
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=tspan, y=S, mode='lines+markers', name='Susceptible'))
    fig.add_trace(go.Scatter(x=tspan, y=E, mode='lines+markers', name='Exposed'))
    fig.add_trace(go.Scatter(x=tspan, y=Ic, mode='lines+markers', name='Infected'))
    fig.add_trace(go.Scatter(x=tspan, y=Isc, mode='lines+markers', name='Sub Infected'))
    fig.add_trace(go.Scatter(x=tspan, y=R, mode='lines+markers',name='Removed'))
    
    if days <= 30:
        step = 1
    elif days <= 90:
        step = 7
    else:
        step = 30
    
    fig.update_layout(title='Simulation of SEIIRS Model',
                       xaxis_title='Day',
                       yaxis_title='Counts',
                       title_x=0.5,
                      width=900, height=600,
                     )
    
    fig.update_xaxes(tickangle=-90, tickformat = None, tickmode='array', tickvals=np.arange(0, days + 1, step))
    fig.show()

In [ ]:
interact(main, 
         initS=IntSlider(min=0, max=100000, step=1, value=initS, description='initS', style=style, layout=slider_layout),
         initE=IntSlider(min=0, max=100000, step=1, value=initE, description='initE', style=style, layout=slider_layout),
         initIc=IntSlider(min=0, max=100000, step=10, value=initIc, description='initIc', style=style, layout=slider_layout),
         initIsc=IntSlider(min=0, max=100000, step=10, value=initIsc, description='initIsc', style=style, layout=slider_layout),
         initR=IntSlider(min=0, max=100000, step=10, value=initR, description='initR', style=style, layout=slider_layout),
         initN=IntSlider(min=0, max=13800000, step=10, value=initN, description='initN', style=style, layout=slider_layout),
         beta=FloatSlider(min=0, max=2, step=0.01, value=beta, description='Infection rate', style=style, layout=slider_layout),
         kappa=FloatSlider(min=0, max=1, step=0.01, value=kappa, description='Exposure to Infection rate', style=style, layout=slider_layout), 
         gamma=FloatSlider(min=0, max=1, step=0.001, value=gamma, description='Removal Rate', style=style, layout=slider_layout),
         delta=FloatSlider(min=0, max=1, step=0.01, value=delta, description='Sus Feedback', style=style, layout=slider_layout),
         roe=FloatSlider(min=0, max=1, step=0.01, value=roe, description='% Clinical', style=style, layout=slider_layout),
         alpha=FloatSlider(min=0, max=1, step=0.01, value=alpha, description='Reduced Infectivity Isc', style=style, layout=slider_layout),
         days=IntSlider(min=1, max=600, step=7, value=days, description='Days', style=style, layout=slider_layout)
        );